In [10]:
import re
import os

In [9]:
dh_journals_cr_dois = open("../../../data/phd_thesis/dh_journals_analysis/dh_journals_cr_dois.csv")

COCI_API_REFS = "https://opencitations.net/index/coci/api/v1/references/%s"

In [ ]:
params = ""
req_call = call%issn_value
print("Calling ... "+req_call+params)
response = requests.get(req_call+params)
res_dois = __dois

if response.status_code == 200:
    json_data = json.loads(response.content)

In [16]:
import datetime
import isodate
d_a = isodate.parse_duration('P6Y11M')
d_b = isodate.parse_duration("P41Y1D")